In [3]:
import requests
import os
import json
from api_keys import BEARER_TOKEN
from tqdm import tqdm
import os
import pandas as pd
from datetime import datetime
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import re


[nltk_data] Downloading package stopwords to /home/julio/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from unicodedata import normalize as norm

def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str).encode('ASCII', 'ignore')
    only_ascii = nfkd_form.encode('ASCII', 'ignore')
    return only_ascii


In [ ]:
unicodedata.normalize('NFKD', "mãe").encode('ASCII', 'ignore')

In [ ]:
norm('NFKD', "mãe").encode('ascii', 'ignore').decode().lower()

In [ ]:
text = "https:4//t.co/lmvoS5uwFd poxa globoplay...  https:3//t.co/lmvoS5uwFd   https:2//t.co/lmvoS5uwFd https:1//t.co/lmvoS5uwFd   "

In [ ]:

regex_remove_https = 'https([a-zA-Zà-úÀ-Ú0-9]|[-()\#/@;:<>{}`+=~|.!?,])+'
#text_without_https = re.sub(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https)," ",text)
#print("\n")
#print("\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https))
#print("\n")

text_without_https = re.sub(r"(\s|^){0}(\s{0})*($|\s)".format(regex_remove_https)," ",text)

In [ ]:
text_without_https

In [ ]:
if re.search(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https),text_without_https):
    
    print("1")

In [ ]:
re.findall(r"\s{}\s|\s{}$|^{}\s".format(regex_remove_https,regex_remove_https,regex_remove_https),text,re.MULTILINE)

# Funções (Análise Exploratória)

### Função para plotar bar plot com a contagem de tokens

In [ ]:
def plot_bar_count_words(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = CountVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para plotar bar plot com tf-idf

In [ ]:
def plot_bar_tf_idf(text_column=None,
                         label_column=None,
                         name_class=None,
                         dataframe=None,
                         metric='SUM',
                         top=50,return_df=True):
    
    corpus = dataframe[text_column].values
    
    vectorizer = TfidfVectorizer()
    data_vect = vectorizer.fit_transform(corpus)
    data_vect = data_vect.toarray()
    
    df_count_words =  pd.DataFrame({
    "WORDS":vectorizer.get_feature_names(),
    "MEAN":data_vect.mean(axis=0),
    "SUM":data_vect.sum(axis=0),
    "STD":data_vect.std(axis=0),
    "MAX":data_vect.std(axis=0)
    }) 
    
    

    if return_df:
    
        return df_count_words[[metric,'WORDS']].sort_values(by=[metric],ascending=False)[0:top]
    
    else:
        
        fig = plt.figure(figsize=(15,10))
        
        ax = sns.barplot(x=metric, 
                 y="WORDS", 
                 data=df_count_words[[metric,'WORDS']].sort_values(by=[metric],
                                                                            ascending=False)[0:top])


### Função para contagem de tokens

In [ ]:
def calculate_number_words(text):

    quantity_of_words = text.split(" ")

    quantity_of_words = [i for i in quantity_of_words if i!=""]

    quantity_of_words = len(quantity_of_words)

    return quantity_of_words


### Função para contagem de diferentes tokens

In [ ]:
def calculate_number_diferent_words(text):

    quantity_of_diferent_words = text.split(" ")

    quantity_of_diferent_words = [i for i in quantity_of_diferent_words if i!=""]

    quantity_of_diferent_words = set(quantity_of_diferent_words)

    quantity_of_diferent_words = list(quantity_of_diferent_words)

    quantity_of_diferent_words = len(quantity_of_diferent_words)

    return quantity_of_diferent_words


### Função para criar textos sem repetição de palavras para ser utilizado na análise exploratória 

In [ ]:
def convert_text_to_no_repeat_words(text):

    text_with_no_repeat_words = text.split(" ")

    text_with_no_repeat_words = [i for i in text_with_no_repeat_words if i!=""]

    text_with_no_repeat_words = set(text_with_no_repeat_words)

    text_with_no_repeat_words = list(text_with_no_repeat_words)

    text_with_no_repeat_words = " ".join(text_with_no_repeat_words)

    return text_with_no_repeat_words

### Função para o pré-processamento do texto 

In [ ]:
def text_cleaner(text):
    
    nltk_stopwords = stopwords.words('portuguese')

    collection_text = [ {"text" : text}]
    text = pd.DataFrame(collection_text)

    text['text'] = text['text'].astype('str')
    text['text'] = text['text'].str.lower()
    text['text'] = text['text'].str.replace('\n',' ')
    text['text'] = text['text'].str.replace('\r',' ')
    text['text'] = text['text'].apply(lambda x: norm('NFKD', x).encode('ascii', 'ignore').decode())
    text['text'] = text['text'].apply(lambda x: re.sub(r'[^a-zA-Z0-9]',' ',x))
    text['text'] = text['text'].apply(lambda x: re.sub(r'\s+',' ',x))
    pat = r'\b(?:{})\b'.format('|'.join(nltk_stopwords))
    text['text'] = text['text'].str.replace(pat,'')
    text = text['text'].values[0]

    return text

### Função para limpeza dos textos

In [ ]:
def text_cleaner(text,stop_words_domain =None):
    
    nltk_stopwords = stopwords.words('portuguese')
    regex_stop_words = '|'.join(nltk_stopwords)
    text = re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s"," ",text)
    text = re.sub(r"[^a-zA-ZÀ-Úà-ú]+"," ",text)
    text = re.sub(r"\s({})\s|\s({})$|^({})\s".format(regex_stop_words,regex_stop_words,regex_stop_words)," ",text)
    
    return text

In [ ]:
txt = " uhahua,não,"

In [ ]:
text_cleaner(txt)

In [ ]:
print(stopwords.words('portuguese'))

# Funções (Extração de Tweets)

In [9]:
# To set your enviornment variables in your terminal run the following line:
# export 'BEARER_TOKEN'='<your_bearer_token>'


def create_url(query = "@globoplay -is:retweet",until_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&until_id={}&tweet.fields=author_id,created_at".format(
            query,until_id
        )
        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&tweet.fields=author_id,created_at".format(
            query
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = "@BBB -is:retweet",until_id=None):
    bearer_token = BEARER_TOKEN
    url = create_url(query,until_id)
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    
    print("\n")
    print(json_response)
    print("\n")
    
    return json_response

def extract_many_tweets(qnt_cycle=10,folder="data_tweets",until_id=None,query="@BBB",start_from_id=None):
    
    
    oldest_id = None
    
    newest_id = None
    
    
    
    for i in tqdm(range(qnt_cycle)):
    
        
        if i == 0:
            
            #extract the 100 tweets first
            
            if not start_from_id:
        
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=None)
            
            else:
                
                data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=until_id)
                
            
            df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
            
            #get the current date
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            
            oldest_id = data_tweets['meta']['oldest_id']
            
            newest_id = data_tweets['meta']['newest_id']
            
            oldest_date = date_extraction
            
            df_data_tweets = df_data_tweets_temp.copy()
            
            # name file
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file,sep=",")
            
    
            
        else:
            
            
            #extract more 100 tweets older
            
            data_tweets_temp = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=oldest_id)
            
            df_data_tweets_temp = pd.DataFrame(data_tweets_temp["data"])
            
            
            #get the current date
            
            
            date_extraction = datetime.now()
            
            df_data_tweets_temp["date_extraction"] = date_extraction 
            
            oldest_id = data_tweets_temp['meta']['oldest_id']
            
            df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])
            
            date_extraction = datetime.now()
            
            df_data_tweets.reset_index(inplace=True,drop=True)
            
            
            # remove old files
            
            os.remove(name_file)
            
            
            # name file
            
            oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
            
            date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
            
            
            name_file = "./{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)
            
            # persist base
            
            df_data_tweets.to_csv(name_file.format(folder),sep=",")
            
            

    
    return df_data_tweets
    
    


In [78]:
import requests
import os
import json
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import time



def create_url(query = None,until_id=None,since_id=None):
    
    #query = "@BBB -is:retweet"
    #"from:twitterdev -is:retweet"
    # Tweet fields are adjustable.
    # Options include:
    # attachments, author_id, context_annotations,
    # conversation_id, created_at, entities, geo, id,
    # in_reply_to_user_id, lang, non_public_metrics, organic_metrics,
    # possibly_sensitive, promoted_metrics, public_metrics, referenced_tweets,
    # source, text, and withheld
    
    if until_id:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&tweet.fields=author_id,created_at&until_id={}&".format(
            query,until_id
        )

    elif since_id:

        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&tweet.fields=author_id,created_at&since_id={}&".format(
            query,since_id
        )

        
    else:
        
        url = "https://api.twitter.com/2/tweets/search/recent?query={}&max_results=10&tweet.fields=author_id,created_at".format(
            query
        )
            
    return url


def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers


def connect_to_endpoint(url, headers):
    response = requests.request("GET", url, headers=headers)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def extract_100_tweets(query = None,until_id=None,since_id=None,key_twitter = None):
    bearer_token = key_twitter
    
    if not until_id:
        
        url = create_url(query,until_id = until_id)
        
    elif not since_id:
        
        url = create_url(query,since_id = since_id)
        
    else:
        
        url = create_url(query = query)
    
    headers = create_headers(bearer_token)
    json_response = connect_to_endpoint(url, headers)
    data_tweets = json.dumps(json_response, indent=4, sort_keys=True)
    return json_response

def extract_many_tweets(qnt_cycle=10,
                        folder="data_tweets",
                        until_id=None,
                        since_id=None,
                        query="@BBB",
                        bearer_token = None,
                        df_concat_path=None):


    if since_id:

        data_tweets_recent = extract_100_tweets(query = "{} -is:retweet".format(query),
                                                since_id=since_id,
                                               key_twitter = bearer_token)
        '''
        time.sleep(1)

        oldest_id = data_tweets_recent['meta']['oldest_id']

        newest_id = data_tweets_recent['meta']['newest_id']

        if df_concat_path:

            df_data_tweets_old = pd.read_csv(df_concat_path)

            df_data_tweets = pd.concat([data_tweets_recent,df_data_tweets_old])

        else:

            df_data_tweets = data_tweets_recent


        name_file = "{}/running/persist_tweets.csv".format(folder)

        # persist base
                
        df_data_tweets.to_csv(name_file,sep=",")
        '''
        
        data = {
                'data':data_tweets_recent["data"],
                'meta': {'newest_id': data_tweets_recent["meta"]["newest_id"],
                        'oldest_id': data_tweets_recent["meta"]["oldest_id"],'query':query}
                }       
        
    else:

        
        oldest_id = None

        newest_id = None
        
        for i in tqdm(range(qnt_cycle)):
        
            
            if i == 0:
                
                #extract the 100 tweets first
                
                if not until_id:
            
                    data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=None,key_twitter = bearer_token)
                    time.sleep(1)
                
                else:
                    
                    data_tweets = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=until_id,key_twitter = bearer_token)
                    time.sleep(1)
                    
                
                df_data_tweets_temp = pd.DataFrame(data_tweets["data"])
                
                #get the current date
                
                
                oldest_id = data_tweets['meta']['oldest_id']

                newest_id = data_tweets['meta']['newest_id']
                
                df_data_tweets = df_data_tweets_temp.copy()
                
                # name file
                
                #date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
                
                #name_file = "{}/persist_tweets_{}_{}.csv".format(folder,date_extraction_str,date_extraction_str)
                
                #name_file = "{}/running/persist_tweets.csv".format(folder)

                # persist base
                
                #df_data_tweets.to_csv(name_file,sep=",")
                
        
                
            else:
                
                
                #extract more 100 tweets older

                data_tweets_temp = extract_100_tweets(query = "{} -is:retweet".format(query),until_id=oldest_id,key_twitter = bearer_token)
                time.sleep(1)
                
                df_data_tweets_temp = pd.DataFrame(data_tweets_temp["data"])
                
                
                #get the current date
                
                
                
                oldest_id = data_tweets_temp['meta']['oldest_id']
                
                df_data_tweets = pd.concat([df_data_tweets,df_data_tweets_temp.copy()])

                
                df_data_tweets.reset_index(inplace=True,drop=True)
                
                
                # remove old files
                
                #os.remove(name_file)
                
                
                # name file
                
                #oldest_date_str = str(oldest_date).replace(".","-").replace(":","-").replace(" ","-")
                
                #date_extraction_str = str(date_extraction).replace(".","-").replace(":","-").replace(" ","-")
                
                
                #name_file = "{}/persist_tweets_{}_{}.csv".format(folder,oldest_date_str,date_extraction_str)

                #name_file = "{}/running/persist_tweets.csv".format(folder)
                
                # persist base
                
                #df_data_tweets.to_csv(name_file.format(folder),sep=",")
                
    
        data = {
                'data':df_data_tweets.to_dict("records"),
                'meta': {'newest_id': newest_id,
                        'oldest_id': oldest_id,'query':query}
                }

    #status_system = {'df_concat_path':name_file,'newest_id':newest_id,'oldest_id':oldest_id,'query':query}

    #f_status_system = open("{}/running/status_system.json".format(folder), 'w')
    #json.dump(status_system, f_status_system)


    return data
    
    


# Extração de Tweets

In [104]:
data_tweets_final = extract_many_tweets(qnt_cycle=1,folder="data_tweets",query="globoplay",bearer_token = BEARER_TOKEN)

100%|██████████| 1/1 [00:01<00:00,  1.91s/it]


In [105]:
f_status_system = open("data_tweets.json", 'w', encoding='utf8')


json.dump(data_tweets_final, f_status_system, ensure_ascii=False)

In [102]:
data_tweets_final

{'data': [{'text': 'n tem nada d bom p fazer na netflix, amazon, globoplay, disney vou cancelar tudo, procuro as coisas pra ver e n acho',
   'author_id': '1358960556246466560',
   'created_at': '2021-05-12T02:51:22.000Z',
   'id': '1392311409795555334'},
  {'text': '@dayane_v333 @HugoGloss Fica tudo no globoplay',
   'author_id': '1274914698358636547',
   'created_at': '2021-05-12T02:51:22.000Z',
   'id': '1392311409258684418'},
  {'text': 'perdi o começo do no limite agora to esperando acabar pra ver a integra no globoplay https://t.co/WcbYuoHQAs',
   'author_id': '1308267423813038080',
   'created_at': '2021-05-12T02:51:18.000Z',
   'id': '1392311390937960451'},
  {'text': 'A entrevista com o eliminado vai ser ao vivo no @Globoplay com a @anaclaraac assim que acabar i programa, igual o BBB  #NoLimite',
   'author_id': '104565437',
   'created_at': '2021-05-12T02:51:13.000Z',
   'id': '1392311370344013825'},
  {'text': 'Minha gente, esse mahmoud é perigosíssimo #NoLimite #globoplay #

In [40]:
data_tweets_final["text"][0]

'@globoplay será que a senhora poderia por favor assinar um contrato com a dona @amazonBR para que eu possa assistir vocês na minha tv? Obrigada, de nada!\U0001f972'

In [79]:
data_tweets_final = extract_many_tweets(qnt_cycle=1,folder="data_tweets",query="globoplay",bearer_token = BEARER_TOKEN,since_id="1392305680640585733")

In [81]:
df = pd.DataFrame.from_dict(data_tweets_final["data"])

In [82]:
df

,text,author_id,id,created_at
0,@globoplay @rafakalimann_ Eu tô é ansiosa pra ...,1246830918360276993,1392306470952312834,2021-05-12T02:31:45.000Z
1,@Guilherme_jbr Assiste pela globoplay,1347276882190995456,1392306418523512840,2021-05-12T02:31:32.000Z
2,😂😂😂😂😂😂😂😂😂 https://t.co/0BmatrpYyW,115168600,1392306417781063680,2021-05-12T02:31:32.000Z
3,@doublecult netflix e globoplay https://t.co/T...,1030610818122113029,1392306387720495106,2021-05-12T02:31:25.000Z
4,@arlisao No GloboPlay bem possível. Acho que t...,81957241,1392306351070666754,2021-05-12T02:31:16.000Z
5,Atualizando o meme: Segure meu B.O. 😂 #NoLimit...,55569724,1392306348101095434,2021-05-12T02:31:15.000Z
6,"@doublecult netflix, amazon e globoplay",1370061745222529028,1392306263233597440,2021-05-12T02:30:55.000Z
7,@AliExpressBR @globoplay Estou achando esse ta...,2685116886,1392306241972678656,2021-05-12T02:30:50.000Z
8,Calango ou Carcará?\n\n#nolimite #calango #car...,791606337738305540,1392306214009286657,2021-05-12T02:30:43.000Z
9,@Luciana_Miranda Vou assistir na Globoplay depois,27770462,1392306185664139265,2021-05-12T02:30:37.000Z


In [58]:
df.to_dict('records')

[{'id': '1392301970585792515',
  'created_at': '2021-05-12T02:13:52.000Z',
  'author_id': '1293754876825153537',
  'text': 'Reprodução/Globoplay https://t.co/I1j5RcqZTP'},
 {'id': '1392301962876555265',
  'created_at': '2021-05-12T02:13:50.000Z',
  'author_id': '382316896',
  'text': 'a dona da globoplay tendo patrocinio da amazon prime pro programa e botando comercial da netflix\n\nunião entre streamings'},
 {'id': '1392301955024838658',
  'created_at': '2021-05-12T02:13:48.000Z',
  'author_id': '1285377843581661186',
  'text': 'globoplay eh muito merda eu PAGO esse cu mas to vendo a programação por um daqueles canal que passa a programação ao vivo pelo youtube'},
 {'id': '1392301932618866690',
  'created_at': '2021-05-12T02:13:43.000Z',
  'author_id': '90660978',
  'text': 'Alguém me explica pq não tá passando no limite na Globoplay?'},
 {'id': '1392301908640083978',
  'created_at': '2021-05-12T02:13:37.000Z',
  'author_id': '1349167491386372097',
  'text': 'A tribo calango acaba de 

In [54]:
data_tweets_final

{'data': [{'id': '1392301970585792515',
   'created_at': '2021-05-12T02:13:52.000Z',
   'author_id': '1293754876825153537',
   'text': 'Reprodução/Globoplay https://t.co/I1j5RcqZTP'},
  {'id': '1392301962876555265',
   'created_at': '2021-05-12T02:13:50.000Z',
   'author_id': '382316896',
   'text': 'a dona da globoplay tendo patrocinio da amazon prime pro programa e botando comercial da netflix\n\nunião entre streamings'},
  {'id': '1392301955024838658',
   'created_at': '2021-05-12T02:13:48.000Z',
   'author_id': '1285377843581661186',
   'text': 'globoplay eh muito merda eu PAGO esse cu mas to vendo a programação por um daqueles canal que passa a programação ao vivo pelo youtube'},
  {'id': '1392301932618866690',
   'created_at': '2021-05-12T02:13:43.000Z',
   'author_id': '90660978',
   'text': 'Alguém me explica pq não tá passando no limite na Globoplay?'},
  {'id': '1392301908640083978',
   'created_at': '2021-05-12T02:13:37.000Z',
   'author_id': '1349167491386372097',
   'text'

In [53]:
df

,id,created_at,author_id,text
0,1392301970585792515,2021-05-12T02:13:52.000Z,1293754876825153537,Reprodução/Globoplay https://t.co/I1j5RcqZTP
1,1392301962876555265,2021-05-12T02:13:50.000Z,382316896,a dona da globoplay tendo patrocinio da amazon...
2,1392301955024838658,2021-05-12T02:13:48.000Z,1285377843581661186,globoplay eh muito merda eu PAGO esse cu mas t...
3,1392301932618866690,2021-05-12T02:13:43.000Z,90660978,Alguém me explica pq não tá passando no limite...
4,1392301908640083978,2021-05-12T02:13:37.000Z,1349167491386372097,A tribo calango acaba de vencer a segunda prov...
5,1392301896916930566,2021-05-12T02:13:34.000Z,3240847114,@Nanda_Loren @globoplay Eles passaram no ao vi...
6,1392301894425616387,2021-05-12T02:13:34.000Z,2157992258,"Olha a concorrente ai, alo globoplay"
7,1392301873118466051,2021-05-12T02:13:28.000Z,999809364,Pq o Globoplay ao vivo não funciona na Bahia? ...
8,1392301831200591875,2021-05-12T02:13:18.000Z,53046167,@boninho @tvglobo @NoLimite Cadê para a Globop...
9,1392301812628312069,2021-05-12T02:13:14.000Z,74212028,"E a internet que acabou aqui em casa, não volt..."


In [15]:
data_tweets_final["text"][101]

'@globoplay Eu ainda não acredito que vocês fizeram isso... \n\nPelo amor de Deus. \n\nEu não vou perder meu tempo. \nE pra mim tomara que perda o investimento. \n\nSó fez maldade... maldosa mesmo.. a fama dela com relacionamentos com pessoas já era ruim aqui fora.. ai mudou do nada. NAO ACREDITO'

In [ ]:
data_tweets_final["text_clean"] = data_tweets_final["text"].apply(lambda x: text_cleaner(text = x,stop_words_domain =None))

In [ ]:
data_tweets_final[["text","text_clean"]]

In [ ]:
re.sub(,data_tweets_final["text"][3]

In [ ]:
data_tweets_final["text"][3]

In [ ]:
test = 'https://t.co/HaKu8sRLkX Melhor pessoa desse bbb '

In [ ]:
re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[-()\"#/@;:<>{}`+=~|.!?,])+\s"," ",test)

In [ ]:
re.sub(r"\shttps([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+$|^https([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s|\shttps([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s"," ",test)

In [ ]:
https([a-zA-Zà-úÀ-Ú0-9]|[^a-zA-Zà-úÀ-Ú0-9])+\s

In [ ]:
def function_to_calc_histogram(x,initial_interval, final_interval,n_bins,indice=False):

    interval = np.linspace(initial_interval, final_interval, num=n_bins)

    for j,i in enumerate(interval):


        if i == interval[len(interval)-1]:

            if x>=i:

                
                if indice:

                    return j
                
                else:
                    
                    return "{}<".format(x)


        else:

            if x>=i and x<interval[j+1]:

                inicial = round(i, 1)

                final = round(interval[j+1],1)

            
                if indice:

                    return j
                
                else:
                    
                    return "[{},{})".format(inicial,final)


# Criação de uma coluna com os textos sem repetição de palavras para ser utilizado na análise exploratória

In [ ]:
data_tweets_final['text_unique_words'] = data_tweets_final['text'].apply(lambda x: convert_text_to_no_repeat_words(x))

# Calculo Número de tokens

In [ ]:
data_tweets_final['number_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_words(x))

# Calculo Número de diferentes tokens

In [ ]:
data_tweets_final['number_diferent_tokens'] = data_tweets_final['text'].apply(lambda x: calculate_number_diferent_words(x))

# Máximo número de tokens

In [ ]:
max_count = data_tweets_final["number_tokens"].max()

In [ ]:
data_tweets_final["number_tokens"].var()

# Mínimo número de tokens

In [ ]:
min_count = data_tweets_final["number_tokens"].min()

## Dados para o histograma 

In [ ]:
data_tweets_final['bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=False))

In [ ]:
data_tweets_final['indices_bins'] = data_tweets_final['number_tokens'].apply(lambda x: function_to_calc_histogram(x,initial_interval = min_count, final_interval = max_count,n_bins = 10,indice=True))

In [ ]:
data_histogram = data_tweets_final.groupby(["bins","indices_bins"]).sum().sort_values(by=["indices_bins"])

In [ ]:
data_histogram.reset_index(drop=False,inplace=True)

In [ ]:
data_histogram

In [ ]:
data_histogram["bins"].tolist()

In [ ]:
data_histogram["number_tokens"].tolist()

# DF top 10 MEAN

In [ ]:
df_report_mean = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [ ]:
df_report_mean["MEAN"].tolist()

In [ ]:
df_report_mean["WORDS"].tolist()

# DF top 10 SUM docs

In [ ]:
df_report_sum = plot_bar_count_words(text_column='text_unique_words',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [ ]:
df_report_sum["P_DOCS"] =  df_report_sum["SUM"]/len(data_tweets_final)

In [ ]:
df_report_sum

# DF top 10 SUM

In [ ]:
df_report_sum_docs = plot_bar_count_words(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='SUM',top=10,return_df=True)

In [ ]:
df_report_sum_docs

# DF top 10 MEAN TF-IDF

In [ ]:
df_report_tfidf_mean = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MEAN',top=10,return_df=True)

In [ ]:
df_report_tfidf_mean

# DF top 10 MAX TF-IDF

In [ ]:
df_report_tfidf_max = plot_bar_tf_idf(text_column='text',
                                                dataframe=data_tweets_final,
                                                metric='MAX',top=10,return_df=True)

In [ ]:
df_report_tfidf_max